# Install libraries

In [1]:
!pip install "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using c

# Import Libraries

In [2]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import torch
from peft import AutoPeftModelForCausalLM
import transformers

print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.35.0


# See GPU is used or not

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

Device  cuda  is being used


# Load dataset

In [4]:
# Dataset
validation_dataset = load_dataset("eli5_category",split="validation1[:500]")

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

# Load model

In [5]:
# download dataset to colab
# ! mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# !kaggle datasets download -d haniyehaghdam/lamma-25000dataset
# !unzip '/content/lamma-25000dataset.zip'

In [11]:
%%time


model_directory = '/content/laama_15000_25000data/checkpoint-900' # Set this to your directory

new_model = AutoPeftModelForCausalLM.from_pretrained(
    model_directory,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 44.5 s, sys: 34.5 s, total: 1min 18s
Wall time: 4min 54s


# Load tokenizer

In [12]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

CPU times: user 70.9 ms, sys: 13.5 ms, total: 84.4 ms
Wall time: 131 ms


# Testing

# Set promt

In [59]:
question_index_to_ask = 127 # It is the row of validation1
prompt = validation_dataset['title'][question_index_to_ask]

In [60]:
print(prompt)

Why is single payer healthcare cheaper overall than its alternatives?


40

# Long answer

In [84]:
question_index_to_ask = 145 # It is the row of validation1
prompt = validation_dataset['title'][question_index_to_ask]

In [85]:
def answer_query(query,temperature=0.6,long_answer = False):

    if long_answer:
      prompt= 'Generatee long answer for this question ' + query

    else:
      prompt= 'Generate short answer for this question ' + query

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=300,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=30,
                             max_length = 100,
                             max_new_tokens=70,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return answer

In [86]:
answer = answer_query(prompt,long_answer=True)

Generatee long answer for this question My extended family is from the southern U.S., while I grew up in the North, but after visiting them for a week or so I begin to speak with a southern accent. What exactly is going on there?

Answer:
> I’m 24, and I’ve been living in the North for 22 years, but I still have a southern accent. I have a strong southern accent, but I can’t remember the last time I’ve been called “Southern” or “Y’all” by a native. I don’t think I’ve ever been called that. I think that’s the difference. It’s not just a matter of the accent, but also the way you speak, the way you think, and the way you act. I’m not sure how much I can speak to this, since I’m not really from the South, but I’ve noticed that people from the South are more likely to use slang, and more likely to use words like y’all and fixin’. They are also more likely to use words like y’all and fixin’. I think you can also tell the difference in the way people talk, like how they pronounce certain wor

In [87]:
# Remove question from answer and remove last uncomplete sentence
print('Let us try to create long answer, please wait\n')

def pretty_asnwer(answer):
    new_answer = answer
    if new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?'):

        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        if new_answer.find('Answer')>-1:
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]
        #print(new_answer)

    else:
        #print('Not complete answer')
        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer





# Check length of sentence to be long:
# while len(new_answer.split(' '))<=150:
#     old_anwer = new_answer
#     new_one = answer_query(validation_dataset['title'][question_index_to_ask],temperature=0.7)
#     new_clean_one = old_anwer+ pretty_asnwer(new_one)
#     new_answer = new_clean_one


# print('Question: \n',prompt)
# print('Long Answer:\n',new_answer)
new_answer = pretty_asnwer(answer)
print(new_answer)


Let us try to create long answer, please wait

> I’m 24, and I’ve been living in the North for 22 years, but I still have a southern accent. I have a strong southern accent, but I can’t remember the last time I’ve been called “Southern” or “Y’all” by a native. I don’t think I’ve ever been called that. I think that’s the difference. It’s not just a matter of the accent, but also the way you speak, the way you think, and the way you act. I’m not sure how much I can speak to this, since I’m not really from the South, but I’ve noticed that people from the South are more likely to use slang, and more likely to use words like y’all and fixin’. They are also more likely to use words like y’all and fixin’. I think you can also tell the difference in the way people talk, like how they pronounce certain words or use certain words.


# Short answer

In [89]:

short_answer = answer_query(prompt,long_answer=False)
short_answer = pretty_asnwer(short_answer)
print('Let us try to create short answer, please wait\n')
print(short_answer)


Both `max_new_tokens` (=70) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


here
Let us try to create short answer, please wait

You're mimicking your family's accent. It's a common phenomenon called "accent convergence." People tend to pick up the accent of those around them, especially if they spend a lot of time with them.  
